In [2]:
!python -m pip install pandas numpy scikit-learn


^C


  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.3-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached joblib-1.5.3-py3-none-any.whl.metadata (5.5 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
    --------------------------------------- 0.3/11.1 MB ? eta -:--:--
    --------------------------------------- 0.3/11.1 MB ? eta -:--:--
   - -------------------------------------- 0.5/11.1 MB 815.7 kB/s eta 0:00:13
   -- ------------------------------------- 0.8/11.1 MB 800.8 kB/s eta 0:00:13
   -- ------------------------------------- 0.8/11.1 MB 800.8 kB/s eta 0:00:13
   --- ------------------------------------ 1.0/11.1 MB 777.6 kB/s eta 0:00:13
   --- ------------------------------------ 1.0/11.1 MB 777.6 kB/s eta 0:00:13
   --- ------------------------------------ 1.0/11.1 MB 777.

In [1]:
# ============================
# CELL 1: Import libraries
# ============================

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score


In [5]:
import sys
print(sys.executable)


c:\Users\Tanaya\Downloads\aadhaar-enrollment-main\aadhaar-enrollment-main\.venv\Scripts\python.exe


In [8]:
# ============================
# CELL 2: Load cleaned datasets
# ============================

enrol = pd.read_csv(
    r"C:\Users\Tanaya\Downloads\aadhaar-enrollment-main\aadhaar-enrollment-main\aadhaar-enrollment\data\processed\enrol_cleaned.csv"
)

demo = pd.read_csv(
    r"C:\Users\Tanaya\Downloads\aadhaar-enrollment-main\aadhaar-enrollment-main\aadhaar-enrollment\data\processed\demo_cleaned.csv"
)


In [9]:
# ============================
# CELL 3: Date conversion and sorting
# ============================

enrol['date'] = pd.to_datetime(enrol['date'])

enrol = enrol.sort_values(
    by=['state', 'district', 'date']
)


In [11]:
enrol.columns = (
    enrol.columns
    .str.strip()
    .str.lower()
    .str.replace(" ", "_")
)
print(enrol.columns)


Index(['date', 'state', 'district', 'pincode', 'age_0_5', 'age_5_17',
       'age_18_greater'],
      dtype='object')


In [12]:
enrol['enrolment'] = (
    enrol['age_0_5'] +
    enrol['age_5_17'] +
    enrol['age_18_greater']
)


In [13]:
# ============================
# CELL 4: Monthly enrolment aggregation
# ============================

monthly_enrol = (
    enrol.groupby(['state', 'district', 'date'])['enrolment']
    .sum()
    .reset_index()
)


In [14]:
# ============================
# CELL 5: Month-over-month growth rate (TARGET)
# ============================

monthly_enrol['prev_enrolment'] = (
    monthly_enrol.groupby(['state', 'district'])['enrolment']
    .shift(1)
)

monthly_enrol['enrolment_growth_rate'] = (
    (monthly_enrol['enrolment'] - monthly_enrol['prev_enrolment']) /
    monthly_enrol['prev_enrolment']
)

monthly_enrol = monthly_enrol.dropna()


In [16]:
print(demo.columns)


Index(['date', 'state', 'district', 'pincode', 'demo_age_5_17', 'demo_age_17'], dtype='object')


In [17]:
# ============================
# CELL 6: Demographic feature preparation
# ============================

demo_features = (
    demo.groupby(['state', 'district'])[
        ['demo_age_5_17', 'demo_age_17']
    ]
    .mean()
    .reset_index()
)


In [20]:
demo_features.head()


,state,district,demo_age_5_17,demo_age_17
0,Maharashtra,Pune,2.22741,29.423204


In [18]:
# ============================
# CELL 7: Merge enrolment and demographics
# ============================

ml_data = pd.merge(
    monthly_enrol,
    demo_features,
    on=['state', 'district'],
    how='left'
)

ml_data = ml_data.dropna()


In [21]:
print(ml_data.columns)


Index(['state', 'district', 'date', 'enrolment', 'prev_enrolment',
       'enrolment_growth_rate', 'demo_age_5_17', 'demo_age_17'],
      dtype='object')


In [27]:
# ============================
# CELL 8: Feature and target selection
# ============================

X = ml_data[
    [
        'prev_enrolment',
        'demo_age_5_17',
        'demo_age_17'
    ]
]

y = ml_data['enrolment']



In [28]:
# ============================
# CELL 9: Train-test split
# ============================

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [29]:
# ============================
# CELL 10: Train Random Forest model
# ============================

model = RandomForestRegressor(
    n_estimators=100,
    random_state=42
)

model.fit(X_train, y_train)


,"n_estimators n_estimators: int, default=100The number of trees in the forest... versionchanged:: 0.22 The default value of ``n_estimators`` changed from 10 to 100 in 0.22.",100
,"criterion criterion: {""squared_error"", ""absolute_error"", ""friedman_mse"", ""poisson""}, default=""squared_error""The function to measure the quality of a split. Supported criteriaare ""squared_error"" for the mean squared error, which is equal tovariance reduction as feature selection criterion and minimizes the L2loss using the mean of each terminal node, ""friedman_mse"", which usesmean squared error with Friedman's improvement score for potentialsplits, ""absolute_error"" for the mean absolute error, which minimizesthe L1 loss using the median of each terminal node, and ""poisson"" whichuses reduction in Poisson deviance to find splits.Training using ""absolute_error"" is significantly slowerthan when using ""squared_error""... versionadded:: 0.18 Mean Absolute Error (MAE) criterion... versionadded:: 1.0 Poisson criterion.",'squared_error'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",None
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",2
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",1
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: {""sqrt"", ""log2"", None}, int or float, default=1.0The number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None or 1.0, then `max_features=n_features`... note:: The default of 1.0 is equivalent to bagged trees and more randomness can be achieved by setting smaller values, e.g. 0.3... versionchanged:: 1.1 The default of `max_features` changed from `""auto""` to 1.0.Note: the search for a split does not stop until at least onevalid partition of the node samples is found, even if it requires toeffectively inspect more than ``max_features`` features.",1.0
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow trees with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsample

In [ ]:
# ============================
# CELL 11: Model evaluation
# ============================

y_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Absolute Error:", mae)
print("R2 Score:", r2)


Mean Absolute Error: 573.8703333333334
R2 Score: -62.43229018616756


In [31]:
# ============================
# CELL 12: Predict growth rate for all districts
# ============================

ml_data['predicted_growth_rate'] = model.predict(X)


In [32]:
# ============================
# CELL 13: Identify high-risk districts
# ============================

risk_districts = (
    ml_data.groupby(['state', 'district'])['predicted_growth_rate']
    .mean()
    .reset_index()
    .sort_values(by='predicted_growth_rate')
)

risk_districts.head(10)


,state,district,predicted_growth_rate
0,Maharashtra,Pune,582.001912
